In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Datasets/Twitter_Data.csv")

In [2]:
df = df[df["category"]!=0]
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [3]:
import re
def remove_special_words(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'[^\w\s\U0001F600-\U0001F64F]','',text) # Remove any special chars but emojis
    return text

df["clean_text"] = df["clean_text"].apply(remove_special_words)
df["clean_text"] = df["clean_text"].str.lower()

In [4]:
df.head()

,clean_text,category
0,when modi promised minimum government maximum ...,-1.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
8,with upcoming election india saga going import...,1.0


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])
padded_sequences = pad_sequences(sequences, padding='post')

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences,
                                                    df['category'],
                                                    test_size=0.2,
                                                    random_state=42)

In [7]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

checkpoint_path = '/content/drive/MyDrive/models/twitter_sentiment_analysis/best_model.h5'


early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True)

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=0)

lr_reduce_callback = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.34,
    patience=2,
    min_delta=1e-3,
    cooldown=2,
    min_lr=1e-7,
    verbose=0)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Dropout

model = Sequential()
model.add(Embedding(
    input_dim=len(tokenizer.word_index) + 1,
    output_dim=256,
    input_length=padded_sequences.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [13]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)
model.compile(  loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'],)

In [14]:
history = model.fit(X_train, y_train,
            epochs=30,
            batch_size=128,
            validation_data=(X_test, y_test),
            callbacks=[early_stopping,lr_reduce_callback,checkpoint_callback])

Epoch 1/30
674/674 [==============================] - ETA: 0s - loss: 0.5799 - accuracy: 0.1150

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


674/674 [==============================] - 736s 1s/step - loss: 0.5799 - accuracy: 0.1150 - val_loss: 0.4107 - val_accuracy: 0.3190 - lr: 0.0010
Epoch 2/30
674/674 [==============================] - 750s 1s/step - loss: -7.6295 - accuracy: 0.4167 - val_loss: -15.8064 - val_accuracy: 0.3971 - lr: 0.0010
Epoch 3/30
674/674 [==============================] - 765s 1s/step - loss: -538.7845 - accuracy: 0.5122 - val_loss: -1190.6796 - val_accuracy: 0.6124 - lr: 0.0010
Epoch 4/30
674/674 [==============================] - 745s 1s/step - loss: -1453.6815 - accuracy: 0.3439 - val_loss: -1822.0823 - val_accuracy: 0.5792 - lr: 0.0010
Epoch 5/30
674/674 [==============================] - 737s 1s/step - loss: -4029.2805 - accuracy: 0.4940 - val_loss: -4849.4048 - val_accuracy: 0.5471 - lr: 0.0010
Epoch 6/30
674/674 [==============================] - 741s 1s/step - loss: -6005.6807 - accuracy: 0.4966 - val_loss: -5542.3149 - val_accuracy: 0.5291 - lr: 3.4000e-04
Epoch 7/30
674/674 [=================

In [15]:
pd.DataFrame(history.history).to_csv("/content/drive/MyDrive/models/twitter_sentiment_analysis/history.csv")